In [1]:
import pandas as pd
from dash_website import RENAME_DIMENSIONS
from dash_website.utils import BLUE_WHITE_RED


subset_method = "union"
correlation_type = "pearson"
dimension = "*"

data_dimension = pd.read_feather(f"../../all_data/xwas/univariate_correlations/correlations/dimensions/correlations_{RENAME_DIMENSIONS.get(dimension, dimension)}.feather").to_dict()